In [1]:
import pandas as pd
from datetime import datetime

# Text mining
import re
import unicodedata

import nltk
nltk.download(['punkt', 'averaged_perceptron_tagger', 'wordnet'])
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

[nltk_data] Downloading package punkt to /home/sebiche/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sebiche/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/sebiche/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Lectura


In [2]:
df_anime = pd.read_csv('../Datos_Enriquecidos/anime_limpios.csv')

In [3]:
df_anime = df_anime.drop('Unnamed: 0', axis = 1)

In [4]:
df_anime

,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Status,Producers,Source,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL,Sec duration,Emision,Cierre
0,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,Finished Airing,"Bandai Visual, Dentsu, Victor Entertainment, T...",Original,PG-13 - Teens 13 or older,2764.0,1795.0,613.0,42829.0,111931.0,https://cdn.myanimelist.net/images/anime/10/19...,1500.0,2002-07-03,2002-12-25
1,8,Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,Finished Airing,"TV Tokyo, Dentsu",Manga,PG - Children,4240.0,5126.0,14.0,6413.0,15001.0,https://cdn.myanimelist.net/images/anime/7/215...,1380.0,2004-09-30,2005-09-29
2,16,Hachimitsu to Clover,8.00,"Comedy, Drama, Romance","Yuuta Takemoto, a sophomore at an arts college...",TV,24.0,Finished Airing,"Dentsu, Genco, Fuji TV, Asmik Ace, Shueisha",Manga,PG-13 - Teens 13 or older,589.0,862.0,4136.0,81747.0,260166.0,https://cdn.myanimelist.net/images/anime/1301/...,1380.0,2005-04-15,2005-09-27
3,17,Hungry Heart: Wild Striker,7.55,"Comedy, Slice of Life, Sports",As the younger brother of Japanese soccer star...,TV,52.0,Finished Airing,UNKNOWN,Manga,PG-13 - Teens 13 or older,1551.0,4212.0,237.0,12960.0,24172.0,https://cdn.myanimelist.net/images/anime/12/49...,1380.0,2002-09-11,2003-09-10
4,18,Initial D Fourth Stage,8.16,"Action, Drama",Takumi Fujiwara finally joins Ryousuke and Kei...,TV,24.0,Finished Airing,"OB Planning, Studio Jack",Manga,PG-13 - Teens 13 or older,393.0,1273.0,1237.0,97878.0,173710.0,https://cdn.myanimelist.net/images/anime/9/105...,1620.0,2004-04-17,2006-02-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23211,55731,Wu Nao Monu,0.00,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,Not yet aired,UNKNOWN,Web manga,PG-13 - Teens 13 or older,0.0,24723.0,0.0,0.0,0.0,https://cdn.myanimelist.net/images/anime/1386/...,0.0,2023-07-04,2024-03-23
23212,55732,Bu Xing Si: Yuan Qi,0.00,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,Not yet aired,UNKNOWN,Web novel,PG-13 - Teens 13 or older,0.0,0.0,0.0,0.0,0.0,https://cdn.myanimelist.net/images/anime/1383/...,0.0,2023-07-27,2024-03-23
23213,55733,Di Yi Xulie,0.00,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,Finished Airing,UNKNOWN,Web novel,PG-13 - Teens 13 or older,0.0,0.0,0.0,0.0,0.0,https://cdn.myanimelist.net/images/anime/1130/...,0.0,2023-07-19,2024-03-23
23214,55734,Bokura no Saishuu Sensou,0.00,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,Finished Airing,UNKNOWN,Original,PG-13 - Teens 13 or older,0.0,0.0,0.0,0.0,0.0,https://cdn.myanimelist.net/images/anime/1931/...,180.0,2022-04-23,2022-12-31


# Texto

## Synopsis

In [5]:
def clean_text(text: str, lemmatizer: WordNetLemmatizer = WordNetLemmatizer()) -> str:
    text = text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    lower = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
    return ' '.join(list( map( lemmatizer.lemmatize, lower.split(' '))))

In [6]:
texto = """Makes a matrix of histograms for each column of a pd.DataFrame

    Args:
        df (pd.DataFrame): Data to plot
        columns (list, optional): Columns to include in the matrix. Defaults to None.
        rows (int, optional): Number of rows for the matrix. Defaults to None.
        cols (int, optional): Number of columns for the matrix. Defaults to None.
    """

lemmatizer = WordNetLemmatizer()
clean_text(texto, lemmatizer)

'make a matrix of histogram for each column of a pddataframe      args         df pddataframe data to plot         column list optional column to include in the matrix default to none         row int optional number of row for the matrix default to none         col int optional number of column for the matrix default to none     '

In [7]:
df_anime['synopsis_len'] = df_anime['Synopsis'].apply(len)
df_anime['synopsis_words'] = df_anime['Synopsis'].apply(lambda x: len(x.replace('\n', ' ').split(' ')))
df_anime['synopsis_lines'] = df_anime['Synopsis'].apply(lambda x: len(x.split('\n')))
df_anime['synopsis_verbosity'] = 1- df_anime['synopsis_words']/df_anime['synopsis_len']

lemmatizer = WordNetLemmatizer()

df_anime['synopsis_cleaned'] = df_anime['Synopsis'].apply( lambda x: clean_text(x, lemmatizer) )

In [8]:
vect = CountVectorizer(ngram_range=(1, 5), stop_words="english", min_df=10, max_features=150)

In [9]:
vect.fit(df_anime['synopsis_cleaned'])

CountVectorizer(max_features=150, min_df=10, ngram_range=(1, 5),
                stop_words='english')

In [10]:
words = pd.DataFrame(data=vect.transform(df_anime['synopsis_cleaned']).todense(),
                     columns=[f"word_{word}" for word in vect.get_feature_names_out()],
                     index=df_anime.index)

In [11]:
words.sum().sort_values(ascending=False)

word_source       6673
word_anime        5436
word_ha           4786
word_wa           4490
word_available    3852
                  ... 
word_different     553
word_dvd           548
word_cat           545
word_lost          542
word_various       532
Length: 150, dtype: int64

In [12]:
df_anime = df_anime.join(words)

In [ ]:
df_anime

## Name

In [14]:
df_anime['name_len'] = df_anime['Name'].apply(len)
df_anime['name_words'] = df_anime['Name'].apply(lambda x: len(x.replace('\n', ' ').split(' ')))
df_anime['name_lines'] = df_anime['Name'].apply(lambda x: len(x.split('\n')))
df_anime['name_verbosity'] = 1- df_anime['name_words']/df_anime['name_len']
df_anime.sample(3)

,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Status,Producers,Source,...,word_way,word_woman,word_work,word_world,word_year,word_young,name_len,name_words,name_lines,name_verbosity
16171,42279,Wind,0.00,Avant Garde,Beginning with a photograph of a pinhole camer...,Movie,1.0,Finished Airing,UNKNOWN,Original,...,0,0,0,0,0,0,4,1,1,0.750000
12016,34758,Momoiro Bouenkyou Anime Edition,5.70,Hentai,Based on the doujin manga.,OVA,1.0,Finished Airing,"MS Pictures, Chippai",Manga,...,0,0,0,0,0,0,31,4,1,0.870968
1852,2140,Aniyome,6.32,Hentai,"All his life, Takashi has been compared to his...",OVA,2.0,Finished Airing,Digital Works,Visual novel,...,1,0,0,0,0,0,7,1,1,0.857143
17642,44877,Modeng Senlin: Dadi Zhi Ge,0.00,Adventure,No description available for this anime.,TV,20.0,Finished Airing,UNKNOWN,Original,...,0,0,0,0,0,0,26,5,1,0.807692
15824,41318,99 Illusion!,5.46,UNKNOWN,Promotional lyric music video of the song 99 I...,Music,1.0,Finished Airing,UNKNOWN,Unknown,...,0,0,0,0,0,0,12,2,1,0.833333


# Tiempo

In [ ]:
df_anime['Cierre'] = df_anime['Cierre']\
    .apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [19]:
df_anime['Emision'] = df_anime['Emision']\
    .apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [20]:
df_anime['running_time'] = df_anime['Cierre'] - df_anime['Emision']
df_anime.sample(3)

,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Status,Producers,Source,...,word_woman,word_work,word_world,word_year,word_young,name_len,name_words,name_lines,name_verbosity,running_time
14882,39560,Donten,5.81,UNKNOWN,Music video directed by Tomoko Ooshima for the...,Music,1.0,Finished Airing,SME Records,Original,...,0,0,0,0,0,6,1,1,0.833333,293 days
9651,29681,Odoroki Ban,4.68,Avant Garde,Independent animation by Furukawa Taku.,Movie,1.0,Finished Airing,"Studio Zero, Takun Manga Box",Unknown,...,0,0,0,0,0,11,2,1,0.818182,364 days
19721,49036,Noratorium,0.00,UNKNOWN,"A chicken lays a series of eggs, each one hatc...",ONA,6.0,Finished Airing,UNKNOWN,Original,...,0,0,0,0,0,10,1,1,0.900000,364 days
13235,36882,Arifureta Shokugyou de Sekai Saikyou,6.70,"Action, Adventure, Fantasy",The ordinary life of 17-year-old otaku Hajime ...,TV,13.0,Finished Airing,"AT-X, Sony Music Communications, Bushiroad, To...",Light novel,...,0,0,2,0,0,36,5,1,0.861111,91 days
19107,47828,Qi Qu Tegong Dui,0.00,UNKNOWN,No description available for this anime.,TV,26.0,Finished Airing,UNKNOWN,Original,...,0,0,0,0,0,16,4,1,0.750000,1594 days


In [21]:
df_anime.dtypes

anime_id                    int64
Name                       object
Score                     float64
Genres                     object
Synopsis                   object
                       ...       
name_len                    int64
name_words                  int64
name_lines                  int64
name_verbosity            float64
running_time      timedelta64[ns]
Length: 180, dtype: object

In [23]:
df_anime['running_time'] = df_anime['running_time'].astype('timedelta64[s]')

In [27]:
df_anime.sample(3)

,anime_id,Name,Score,Genres,Synopsis,Type,Episodes,Status,Producers,Source,...,word_woman,word_work,word_world,word_year,word_young,name_len,name_words,name_lines,name_verbosity,running_time
3158,3839,Himitsu no Akko-chan 3,6.41,UNKNOWN,"In the Kagami household, the old Western palm-...",TV,44.0,Finished Airing,UNKNOWN,Unknown,...,0,0,0,0,0,22,4,1,0.818182,329 days
16483,42969,Menghuan Shuyuan 4th Season,0.00,"Comedy, Fantasy",Fourth season of Menghuan Shuyuan.,ONA,14.0,Finished Airing,UNKNOWN,Game,...,0,0,0,0,0,27,4,1,0.851852,91 days
18225,45958,Long Taizi,0.00,Fantasy,No description available for this anime.,TV,26.0,Finished Airing,UNKNOWN,Original,...,0,0,0,0,0,10,2,1,0.800000,5926 days
